In [1]:
# Set up the package development environment (e.g. after restarting kernel)
# pwd() should be "C:\\GitWorking\\Autologistic"
using Pkg  
Pkg.activate(pwd())   #At cmd line, could use ]activate .
using Revise
using Autologistic

┌ Info: Recompiling stale cache file C:\Users\krams\.julia\compiled\v0.7\Autologistic\Gemat.ji for Autologistic [05505b62-b0bd-11e8-1cec-1f3e16cbc293]
└ @ Base loading.jl:1185


In [7]:
Pkg.status()

Project Autologistic v0.1.0
    Status `Project.toml`
  [093fc24a] LightGraphs v1.1.0
  [295af30f] Revise v0.7.11
  [10745b16] Statistics 


In [11]:
using LightGraphs
G = Graph(10,20)
P = SimplePairwise(G)

10×10 SimplePairwise{Int64}:
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0